In [1]:
import sys
import pathlib
sys.path.append(str(pathlib.PurePath(pathlib.Path.cwd().parent)))

import os
import multiprocessing
import time
import h5py
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

import dimod
import dwave
import dwave.system
from dwave.system import DWaveSampler, EmbeddingComposite, FixedEmbeddingComposite
import dwave.inspector
import dwave_networkx as dnx
import minorminer

from src.particle_funcs import distance_matrix as distance_matrix
from src.particle_funcs import io as particles_io
import src.leap_funcs.qubo.q_matrix as q_matrix

from src import leap_funcs as leap_funcs
from src.leap_funcs import embedding_quality

from src import h5py_funcs
from src.h5py_funcs import discoveries, init_custom_getstates, io, parameterstudy_using_info_file

Custom getstate functions for dwave.cloud.config.models.ClientConfig, dwave.cloud.client.qpu.Client, dwave.cloud.solver.StructuredSolver, dwave.system.samplers.dwave_sampler.DWaveSampler, dwave.system.composites.embedding.FixedEmbeddingComposite have been initialized.


In [2]:
#folder_path_main = 'test_multi_param_emb_thr'
#info_file_name = 'info.h5'
#folder_embeddings = 'embeddings'
#embeddings_meta_file_name = '00_meta'
#folder_results = 'results'
#
#meta_file_name_path = os.path.join(folder_path_main, folder_embeddings, embeddings_meta_file_name)


In [3]:
def read_info_file(file_name_path='', infoset_name: str = ''):
    return h5py_funcs.io.read_info_from_hdf5_file(file_name_path=file_name_path, infoset_name=infoset_name)

def read_embeddings(reread_info_file = {}, folder_path_main=''):
    import ast
    embeddings = {}
    for key, file_names in reread_info_file['info']['embs']['file_names'].items():
        #print(key, file_names)
        for ind, file in enumerate(file_names['data']):
            file = file.decode('utf-8')
            file_name_path_emb = os.path.join(folder_path_main,'embeddings', file)
            emb = h5py_funcs.io.read_info_from_hdf5_file(
                file_name_path=file_name_path_emb, infoset_name='embedding_mm_01/embedding') 
            emb = {ast.literal_eval(var): list(qs['data']) for var, qs in emb.items()}
            embeddings.update({file: emb})
    return embeddings

In [4]:
def _find_embedding(rocky_analysis_path='', nearest_neighbours=0, folder_name_embs='', file_name_emb=''):

    #folder_path = pathlib.Path(r'C:\zz_tmp_OAH\perf_sphere\qdem_sub_1\01_Particles_0005_CPU_4_GPU_0.rocky.files')
    folder_path = pathlib.Path(rocky_analysis_path)
    print(folder_path)
    dem_data = particles_io.read_dem_data(folder_path=folder_path)

    time_values_for_snapschots = (57.5, 60)
    filenames_for_snapshots = []
    num_particles = None
    particles_coords_names = None
    for key, value in dem_data.items():
        if key not in ('num_particles', 'particle_index'):
            pass
        else:
            continue
        #print(value['attrs']['time_value'][0])
        if np.isclose(value['attrs']['time_value'][0], time_values_for_snapschots).any():
            print(value['attrs']['time_index'], value['attrs']['time_value'])
            print(key)
            filenames_for_snapshots.append(key)
    num_particles = dem_data['num_particles'].astype(int)
    particles_coords_names = dem_data[filenames_for_snapshots[0]]['Particles']['particles_position']['data'].dtype.names
    print(filenames_for_snapshots, num_particles)
    print(particles_coords_names)
    print(type(num_particles))

    part_coords_n = dem_data[filenames_for_snapshots[1]]['Particles']['particles_position']['data'].view((np.double, len(particles_coords_names)))
    part_coords_nm1 = dem_data[filenames_for_snapshots[0]]['Particles']['particles_position']['data'].view((np.double, len(particles_coords_names)))
    #part_coords_n

    distances = distance_matrix.calc_phi_ij(part_coords_n, part_coords_nm1)
    #distances

    Q_dist_diag = q_matrix.q_dist_diag(distances)
    Q_part = q_matrix.q_part(np.shape(distances)[0])
    Q_pos = q_matrix.q_pos(np.shape(distances)[0])
    Q_array = Q_dist_diag + Q_part + Q_pos
    Q_dict = q_matrix.Q_convert_to_dict(Q_array)
    del Q_dict
    Q_dict_new = q_matrix.Q_convert_to_dict_new_keys_iter(Q_array, num_particles)
    if num_particles != nearest_neighbours:
        Q_dict_new = q_matrix.reduce_dict_to_nearest_neighbours(Q_dict_new, distances, nearest_neighbours)

    with open('../API_Token_Oliver_Dev.txt') as file:
        token = file.readline().rstrip()
        architecture = file.readline().rstrip()

    sampler = DWaveSampler(token = token, architecture='pegasus', region='eu-central-1')
    tmp_not_needed_as_a_variable = sampler.adjacency # required for sampler having all data needed for __getstate__, no idea why this is necessary
    sampler_graph = sampler.to_networkx_graph()

    sampler_dict = {}
    _tmp = sampler.client.config.dict().copy()
    _tmp.update({'token': 'removed_for_privacy'})
    sampler_dict.update({'client_reduced': _tmp.copy()})
    #sampler_dict.update({'solver_reduced': 
    _tmp = sampler.solver.data.copy()
    for key in list(_tmp['properties'].keys()):
        if key not in ('num_qubits', 'qubits', 'couplers', 'chip_id', 'tags', 'topology', 'category'):
            _tmp['properties'].pop(key)
    sampler_dict.update({'solver_reduced': _tmp.copy()})
    del _tmp

    logic_vars = set(elem[0] for elem in list(Q_dict_new.keys()))
    layout_source = {elem:tuple(part_coords_n[elem[0]-1][:2]) for elem in logic_vars}
    #print(logic_vars)
    #print(layout_source)


    kwargs_diffusion_candidates = {}
    #kwargs_diffusion_candidates_01 = {
    #    'tries':20, 
    #    'verbose':1,
    #    'layout':layout_source,
    #    #'vicinity':3,
    #    #'viscosity':,
    #    #'delta_t':,
    #    #'d_lim':,
    #    #'downscale':,
    #    #'keep_ratio':,
    #    #'expected_occupancy':
    #    }
    #kwargs_diffusion_candidates_02 = kwargs_diffusion_candidates_01.copy()
    #kwargs_diffusion_candidates_02.update({'tries':50})
    #kwargs_diffusion_candidates = {'mm_01': None, 'mm_02': None, 'em_01':kwargs_diffusion_candidates_01, 'em_02':kwargs_diffusion_candidates_02}
    kwargs_diffusion_candidates = {}
    kwargs_minorminer = {}
    kwargs_minorminer_mm_01 = {
    #        'max_no_improvement':250,
    #        #random_seed=None,
    #        #timeout=1000,
    #        #max_beta=None,
    #        'tries':250,
    #        #inner_rounds=None,
    #        'chainlength_patience':250,
    #        #max_fill=None,
    #        #threads=1,
    #        #return_overlap=False,
    #        #skip_initialization=False,
    #        #verbose=0,
    #        #interactive=False,
    #        #initial_chains=(),
    #        #fixed_chains=(),
    #        #restrict_chains=(),
    #        #suspend_chains=(),
    }
    #kwargs_minorminer_mm_02 = kwargs_minorminer_mm_01.copy()
    #kwargs_minorminer_mm_02.update({'max_no_improvement':500, 'tries':500, 'chainlength_patience':500})
    #kwargs_minorminer = {'mm_01':kwargs_minorminer_mm_01, 'mm_02':kwargs_minorminer_mm_02}
    kwargs_minorminer = {'mm_01':kwargs_minorminer_mm_01}

    kwargs_embera = {}
    #kwargs_embera.update({'em_01': None, 'em_02': None})

    timings_candidates = {}
    timings_embedding = {}
    #kwargs_wo_layout = kwargs_diffusion_candidates_01.copy()
    #kwargs_wo_layout.pop('layout')
    #timings_candidates.update({'mm_01':None, 'mm_02':None, 'em_01':None, 'em_02':None})



    embeddings = {}
    for key in kwargs_minorminer.keys():
        tic = time.time()
        embedding_mm = minorminer.find_embedding(S=Q_dict_new, T=sampler_graph, interactive=True, verbose=1, **kwargs_minorminer[key])
        toc = time.time()
        timings_embedding[key] = toc-tic
        embeddings[key] = {'embedding': embedding_mm, 'sampler': sampler_dict}
    #tic = time.time()
    #candidates_em_layout = embera.preprocess.diffusion_placer.find_candidates(Q_dict_edgelist, sampler_graph, **kwargs_diffusion_candidates)
    #toc = time.time()
    #timings_candidates['em_layout'] = toc-tic



    composites_fixed = {}
    for key, emb_sam in embeddings.items():
        sampler = DWaveSampler(token = token, architecture='pegasus', region='eu-central-1')
        tmp_not_needed_as_a_variable = sampler.adjacency # required for sampler having all data needed for __getstate__, no idea why this is necessary
        composites_fixed[key] = FixedEmbeddingComposite(sampler, emb_sam['embedding'])

    params_sampling = {'label' : 'superdupernice label',
              'annealing_time': 20, 
              'num_reads': 1000, 
              'answer_mode': 'raw', 
              'programming_thermalization': 1000, 
              'readout_thermalization': 0
              }

    for key, value in composites_fixed.items():
        emb = value.embedding
        #print(emb)
        num_qubits = len(set(inner for outer in emb.values() for inner in outer))
        estimate_time_ys = value.child.solver.estimate_qpu_access_time(num_qubits=num_qubits, **params_sampling)
        print('{:22s}'.format(key), num_qubits, estimate_time_ys*1e-6)



    #folder_path = 'test_embeddings'
    #file_name = 'embeddings'
    file_name_path = os.path.join(folder_name_embs, file_name_emb)


    kwargs_file_writing_meta={'file_name_path': file_name_path,
                         'overwrite_data_in_file': False,
                         'track_order': True}

    meta_to_write = {
            'kwargs_diffusion_candidates': kwargs_diffusion_candidates, 
            'kwargs_minorminer': kwargs_minorminer,
            'kwargs_embera': kwargs_embera,
            'timings_candidates': timings_candidates,
            'timings_embedding': timings_embedding
            }
    h5py_funcs.io.write_to_hdf5_file(dict_data=meta_to_write, data_name='embedding', name_suffix='_00_meta', **kwargs_file_writing_meta)

    for key, emb in embeddings.items():
        print(emb)
        emb_to_write = emb
        meta_to_write_emb = {key_meta: value_meta[key] for key_meta, value_meta in meta_to_write.items() if key in value_meta}
        kwargs_file_writing_embs = kwargs_file_writing_meta.copy()
        #kwargs_file_writing_embs.update({'file_name_path': os.path.join(folder_path, key+'.h5')})
        h5py_funcs.io.write_to_hdf5_file(dict_data=emb_to_write, data_name='embedding', name_suffix='_'+key, **kwargs_file_writing_embs)
        h5py_funcs.io.write_to_hdf5_file(dict_data=meta_to_write_emb, data_name='embedding', name_suffix='_'+key+'_meta', **kwargs_file_writing_embs)


In [5]:
def _main_find_embedding(reread_info_file={}, folder_path_main=''):
    for key, dem in reread_info_file['info']['DEMs']['rocky_files'].items():
        rocky_files_path = dem['data'].decode('utf-8')+r'.files'
        print(rocky_files_path)
        for ind, nearest_neighbours in enumerate(reread_info_file['info']['nearest_neighbours'][key]['data']):
            file_name_emb = reread_info_file['info']['embs']['file_names'][key]['data'][ind].decode('utf-8')
            print(file_name_emb)
            folder_name_embs = os.path.join(folder_path_main,'embeddings')
            _find_embedding(rocky_analysis_path=rocky_files_path, nearest_neighbours=nearest_neighbours, folder_name_embs=folder_name_embs, file_name_emb=file_name_emb)
    

In [6]:
def overloaded_submitter_work(self, problem, verbose=0, print_prefix=''):
    print(print_prefix + 'start working on problem {}', problem)
    identifier = [problem.keys()][0]
    embedding=problem['embedding']
    sampler = DWaveSampler(**self.solver)
    tmp_not_needed_as_a_variable = sampler.adjacency # required for sampler having all data needed for __getstate__, no idea why this is necessary
    composite = FixedEmbeddingComposite(child_sampler=sampler, embedding=embedding)
    num_qubits = len(set(inner for outer in embedding.values() for inner in outer))
    estimate_time_ys = composite.child.solver.estimate_qpu_access_time(num_qubits=num_qubits, **problem['kwargs_sampling'])
    _tmp = np.ceil(estimate_time_ys, 1e6)
    num_reads = [problem['kwargs_sampling']['num_reads']]
    if _tmp > 1:
        print(print_prefix + 'problem {} will take too long ({}) to solve'.format(problem, estimate_time_ys))
        problem['num_runs']
        n, r = divmod(num_reads, _tmp)
        num_reads = [n]*_tmp
        num_reads[0] += r
    print(print_prefix + 'problem {} will be solved {} times'.format(problem, num_reads))
    dict_data = {'set_identifier': identifier, 'composite': composite.__getstate__().copy()}
    
    self.lock_info_file.acquire()
    h5py_funcs.parameterstudy_using_info_file.update_timestamp_in_info_file(file_name_path=os.path.join(self.folder_path_main, self.info_file_name), 
                    info_set='info', set_identifier=identifier, name='start', 
                    timestamp=h5py_funcs.parameterstudy_using_info_file._current_datetime_as_string())
    self.lock_info_file.release()

    samples = {}
    is_not_first_runread = False
    num_runs = problem['num_runs']
    for run in range(num_runs):
        print(print_prefix + 'start run {} for problem {}'.format(run, problem))
        for read in num_reads:
            _ans = composite.sample_qubo(**problem['kwargs_sampling'].update({'num_reads': read}))
            _ans = _ans.to_serializable(pack_samples=False)
            if is_not_first_runread:
                _ans.pop('embedding_context')
            else:
                pass
            samples.update({'{:0{width}n}_{:0{width2}n}'.format(run,read,width=len(str(num_runs)),width2=len(str(num_reads))) : _ans})
            is_not_first_runread = True
        print(print_prefix + 'finished run {} for problem {}'.format(run, problem))
    
    self.lock_info_file.acquire()
    h5py_funcs.parameterstudy_using_info_file.update_timestamp_in_info_file(file_name_path=os.path.join(self.folder_path_main, self.info_file_name), 
                    info_set='info', set_identifier=identifier, name='finish', 
                    timestamp=h5py_funcs.parameterstudy_using_info_file._current_datetime_as_string())
    self.lock_info_file.release()

    dict_data.update({'samples': samples})

    kwargs_write_answer = {'file_name_path': os.path.join(self.folder_path_main, 'samples', identifier+'.h5'),
                            'dict_data': dict_data.copy(),
                            'data_name': 'sampleset',
                            'name_suffix': '', 
                            'overwrite_data_in_file': False,
                            'track_order': True}
    print(print_prefix + 'succesfully retrieved answer for problem {}', problem)
    return kwargs_write_answer


In [7]:
def overloaded_writer_work(self, answer, verbose=0, print_prefix=''):
    print(print_prefix + 'start writing answer for problem {}', answer['dict_data']['set_identifier'])
    h5py_funcs.io.write_to_hdf5_file(**answer)
    print(print_prefix + 'finished writing answer for problem {}', answer['dict_data']['set_identifier'])
    return

In [8]:
def child_process_target(**kwargs):
    cpn = multiprocessing.current_process().name
    print(print_prefix + cpn, 'started with inputs', kwargs['target'])
    #answer = overloaded_submitter_work(*kwargs['submitter']['args'], **kwargs['submitter']['kwargs'])
    #overloaded_writer_work(*((answer,) + kwargs['writer']['args']), **kwargs['writer']['kwargs'])

    st = leap_funcs.qubo.parameterstudy.Multithread_Variationstudy(
        num_threads_submitters=5, num_threads_writers=5
    )
    st.submitter_work = overloaded_submitter_work
    st.writer_work = overloaded_writer_work
    st.folder_path_main = kwargs['folder_path_main']
    st.info_file_name = kwargs['info_file_name']
    st.problems = kwargs['submitter']['args'][0]
    st.solver.update(**kwargs['target']['kwargs_dwavesampler'])
    st.start_execution(verbose=0)


In [9]:
def _main_create_chunks(arg={}, num_chunks=0):
    splitter = np.array_split(list(arg.keys()), num_chunks)
    chunks = [arg[spl[i]] for spl in splitter for i in range(len(spl))]
    return chunks

In [10]:
def main():
    folder_path_main = os.path.join(os.getcwd(), '01_out', 'sub_1')
    info_file_name = 'study_nearest_neighbours_info.h5'
    #_main_create_info_file(folder_path_main=folder_path_main, info_file_name=info_file_name)
    reread_info_file = read_info_file(os.path.join(folder_path_main, info_file_name), infoset_name='')
    
    #_main_find_embedding(reread_info_file=reread_info_file, folder_path_main=folder_path_main)
    reread_embeddings = read_embeddings(reread_info_file=reread_info_file, folder_path_main=folder_path_main)
    sampler_params = {id.decode('utf-8'): 
                        {'embedding': list(reread_embeddings.keys())[i],
                         'num_runs': 10,
                         'kwargs_sampling': {'num_reads':1000}
                         }
                    for i, id in enumerate(reread_info_file['info']['study']['data']['identifiers'][:len(reread_embeddings)])}
    
    iterations = len(sampler_params)
    chunk_size = 50
    num_chunks = np.ceil(iterations/chunk_size).astype(int)
    chunks = _main_create_chunks(sampler_params, num_chunks)

    multiprocessing.set_start_method('spawn')
    print('num_chunks =', num_chunks)
    print(__name__)
    
    with open('../API_Token_Dev.txt', mode='rt') as file:
        token = file.readline().rstrip()
        #kwargs_dwavesampler = {'token': token, 'architecture': 'pegasus', 'region': 'eu-central-1'}

    for chunk_id, chunk in enumerate(chunks[0:2]):
        print('chunk', chunk_id+1, 'of', num_chunks)
        #p = multiprocessing.Process(target=test_function.f_test)
        kwargs_dwavesampler = {'token' : token, 'region':'eu-central-1', 'architecture':'pegasus', 'name':'Advantage_system5.4'}
        kwargs_target = {'print_prefix': ' ', 'kwargs_dwavesampler': kwargs_dwavesampler, 'folder_path_main': folder_path_main, 'info_file_name': info_file_name}
        inputs_submitter = {'args': chunk, 'kwargs':{'print_prefix': ' '}}
        inputs_writer = {'args': (), 'kwargs':{'print_prefix': ' '}}
        inputs_target = {'args': (), 'kwargs':{'target': kwargs_target, 'submitter': inputs_submitter, 'writer': inputs_writer}}

        p = multiprocessing.Process(target=child_process_target, args=inputs_target['args'], kwargs=inputs_target['kwargs'])
        p.start()
        p.join()
        p.close()
    
    
    
    
    
    return sampler_params

if __name__ == '__main__':
    print('This is the main process')
    main()


This is the main process


num_chunks = 1
__main__
chunk 1 of 1
chunk 2 of 1
